## ハフマン符号を生成する


In [17]:
import heapq
from collections import defaultdict


class Node:
    def __init__(self, char, freq):
        self.char = char
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


def calculate_frequency(s):
    total = len(s)
    frequency = defaultdict(int)

    for char in s:
        frequency[char] += 1

    for char, freq in frequency.items():
        frequency[char] = freq / total * 100  # Convert to percentage

    return frequency


def build_huffman_tree(frequency):
    heap = [Node(char, freq) for char, freq in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)

        merged = Node(None, left.freq + right.freq)
        merged.left = left
        merged.right = right

        heapq.heappush(heap, merged)

    return heap[0]


def generate_huffman_codes(node, code="", mapping=None):
    if mapping is None:
        mapping = {}

    if node is not None:
        if node.char is not None:
            mapping[node.char] = code

        generate_huffman_codes(node.left, code + "0", mapping)
        generate_huffman_codes(node.right, code + "1", mapping)

    return mapping


def calculate_compression_ratio(frequency, huffman_codes):
    huffman_length = sum(
        frequency[char] * len(huffman_codes[char]) for char in huffman_codes
    )
    fixed_length = 2 * 100  # 2 bits for each character, and the total frequency is 100%
    return (huffman_length / fixed_length) * 100


def encode_string(s, huffman_codes):
    return sum(len(huffman_codes[char]) for char in s)


def huffman_coding(frequency):
    huffman_tree = build_huffman_tree(frequency)
    huffman_codes = generate_huffman_codes(huffman_tree)
    return huffman_codes

In [18]:
##各文字とその出現頻度を渡すと、ハフマン符号を返す関数
if __name__ == "__main__":
    frequency = {"A": 15, "G": 25, "C": 20, "T": 40}
    huffman_codes = huffman_coding(frequency)
    print("各文字と頻度:", frequency)
    print("ハフマンコード:", huffman_codes)
    ##ハフマンコードで文字列をエンコードすると、各文字に2bitずつ割り当てた場合の約何%の長さとなるかを返す関数
    compression_ratio = calculate_compression_ratio(frequency, huffman_codes)
    print(f"Compression ratio compared to 2-bit fixed length: {compression_ratio:.2f}%")

各文字と頻度: {'A': 15, 'G': 25, 'C': 20, 'T': 40}
ハフマンコード: {'T': '0', 'G': '10', 'A': '110', 'C': '111'}
Compression ratio compared to 2-bit fixed length: 97.50%


In [19]:
##文字列を与えるとその文字列のハフマン符号を返す
if __name__ == "__main__":
    s = "SUMOMOMO_MOMO_MOMOMO_MOMO"
    frequency = calculate_frequency(s)
    huffman_codes = huffman_coding(frequency)
    ##元の文字列を生成した符号でエンコードした結果の bit 長を返す関数
    encoded_length = encode_string(s, huffman_codes)
    print("各文字と頻度:", frequency)
    print("ハフマンコード:", huffman_codes)
    print(f"Encoded string length: {encoded_length} bits")

各文字と頻度: defaultdict(<class 'int'>, {'S': 4.0, 'U': 4.0, 'M': 40.0, 'O': 40.0, '_': 12.0})
ハフマンコード: {'O': '0', 'U': '1000', 'S': '1001', '_': '101', 'M': '11'}
Encoded string length: 47 bits


文字に符号の指定がある場合


In [22]:
def huffman_coding_with_fixed_code(frequency, fixed_char, fixed_code):
    total_freq_excluding_fixed = sum(
        freq for char, freq in frequency.items() if char != fixed_char
    )
    adjusted_frequency = {
        char: (freq / total_freq_excluding_fixed) * 100
        for char, freq in frequency.items()
        if char != fixed_char
    }

    # ハフマンツリーを構築
    huffman_tree = build_huffman_tree(adjusted_frequency)

    # ハフマンコードを生成
    huffman_codes = generate_huffman_codes(huffman_tree)

    # 固定されたコードを追加
    huffman_codes[fixed_char] = fixed_code

    return huffman_codes


# 各文字とその出現頻度
frequency = {"A": 15, "G": 25, "C": 20, "T": 40}

# ハフマン符号の生成（Aに000を割り当てる）
huffman_codes = huffman_coding_with_fixed_code(frequency, "A", "000")
print("ハフマンコード:", huffman_codes)
compression_ratio = calculate_compression_ratio(frequency, huffman_codes)
print(f"Compression ratio compared to 2-bit fixed length: {compression_ratio:.2f}%")

ハフマンコード: {'T': '0', 'C': '10', 'G': '11', 'A': '000'}
Compression ratio compared to 2-bit fixed length: 87.50%
